In [ ]:
import requests
from pathlib import Path
from shapely.geometry import shape
import json
import isce2_topsapp
from isce2_topsapp.delivery_prep import prepare_for_delivery
from isce2_topsapp.water_mask import get_water_mask_dataframe, get_water_mask_raster
from isce2_topsapp.delivery_prep import get_dataset_schema
import rasterio
import matplotlib.pyplot as plt
import jsonschema

# Prepare for Delivery

In [ ]:
url = 'https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-064-tops-20210723_20210711-015001-35393N_33512N-PP-6267-v2_0_4.nc'
nc_path = Path(url.split('/')[-1])

resp = requests.get(url)
with open(nc_path, 'wb') as file:
    file.write(resp.content)

In [ ]:
sample_data_file = Path(isce2_topsapp.__file__).parent.parent / 'tests' / 'sample_loc_metadata.json'
sample_data = json.loads(sample_data_file.read_text())
sample_data['orbit_directory'] = Path(sample_data['orbit_directory'])
sample_data['reference_orbits'] = [Path(orbit) for orbit in sample_data['reference_orbits']]
sample_data['secondary_orbits'] = [Path(orbit) for orbit in sample_data['secondary_orbits']]
sample_data['full_res_dem_path'] = Path(sample_data['full_res_dem_path'])
sample_data['low_res_dem_path'] = Path(sample_data['low_res_dem_path'])

sample_data['gunw_geo'] = shape(sample_data['gunw_geo'])

In [ ]:
final_prod_directory = prepare_for_delivery(nc_path, sample_data)
paths = list(final_prod_directory.glob('*'))
paths

# Validate JSON

In [ ]:
schema = get_dataset_schema()
metadata = json.load(open('S1-GUNW-A-R-064-tops-20210723_20210711-015001-35393N_33512N-PP-6267-v2_0_4/S1-GUNW-A-R-064-tops-20210723_20210711-015001-35393N_33512N-PP-6267-v2_0_4.json'))

assert(jsonschema.validate(instance=metadata, schema=schema) is None)

# Water Mask Tests

In [ ]:
nc_path_new = list(filter(lambda x: '.nc' in x.name, paths))[0]

In [ ]:
with rasterio.open(f'netcdf:{str(nc_path_new)}:/science/grids/data/connectedComponents') as ds:
    p = ds.profile

In [ ]:
%%time

X = get_water_mask_raster(p)

In [ ]:
plt.imshow(X)

Get a profile over the dateline

In [ ]:
t = p['transform']
t2 = t.translation(xoff=-61, yoff=31) * t
t2

In [ ]:
%%time

p2 = p.copy()
p2['transform'] = t2
X2 = get_water_mask_raster(p2)

In [ ]:
plt.imshow(X2)

## Read Vectorfile

Again over the dateline in the +180 degree region.

In [ ]:
bounds = [175, 64, 185, 66]

df = get_water_mask_dataframe(bounds)

In [ ]:
df.plot()
